# Gluonを使ってみる

In [1]:
import mxnet as mx
from mxnet import gluon, nd, autograd, ndarray
import numpy as np

In [2]:
train_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=True, transform=lambda data, label: (data.astype(np.float32)/255, label)),
                                      batch_size=32, shuffle=True)
test_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=False, transform=lambda data, label: (data.astype(np.float32)/255, label)),
                                     batch_size=32, shuffle=False)

In [3]:
net = gluon.nn.Sequential()

with net.name_scope():
    net.add(gluon.nn.Dense(128, activation="relu")) 
    net.add(gluon.nn.Dense(64, activation="relu")) 
    net.add(gluon.nn.Dense(10)) 

In [4]:
ctx = mx.cpu()

In [5]:
net.collect_params().initialize(mx.init.Normal(sigma=0.05))
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .1})

In [6]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for i, (data, label) in enumerate(data_iterator):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

In [14]:
epochs = 10
smoothing_constant = .01
for e in range(epochs):
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx).reshape((-1, 784))
        label = label.as_in_context(ctx)
        with autograd.record(): 
            output = net(data) 
            loss = softmax_cross_entropy(output, label)
            loss.backward()
        trainer.step(data.shape[0])
        
        ##########################
        #  移動平均
        ##########################
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = (curr_loss if ((i == 0) and (e == 0))
                       else (1 - smoothing_constant) * moving_loss + (smoothing_constant) * curr_loss)

    test_accuracy = evaluate_accuracy(test_data, net)
    train_accuracy = evaluate_accuracy(train_data, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" % (e, moving_loss, train_accuracy, test_accuracy))

Epoch 0. Loss: 0.183132073529, Train_acc 0.956833333333, Test_acc 0.9566
Epoch 1. Loss: 0.109478868632, Train_acc 0.972666666667, Test_acc 0.967
Epoch 2. Loss: 0.0842953289678, Train_acc 0.982433333333, Test_acc 0.9746
Epoch 3. Loss: 0.0693212556551, Train_acc 0.985033333333, Test_acc 0.9738
Epoch 4. Loss: 0.0513696662667, Train_acc 0.985366666667, Test_acc 0.9732
Epoch 5. Loss: 0.0467174945326, Train_acc 0.9903, Test_acc 0.9774
Epoch 6. Loss: 0.0376790400984, Train_acc 0.991616666667, Test_acc 0.9772
Epoch 7. Loss: 0.0315229436829, Train_acc 0.994066666667, Test_acc 0.9779
Epoch 8. Loss: 0.0327898717586, Train_acc 0.990283333333, Test_acc 0.9718
Epoch 9. Loss: 0.0223681131653, Train_acc 0.99335, Test_acc 0.9779
